In [1]:
import pandas as pd
import ast
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
rating_data = pd.read_csv('./output/user_list.csv')
movie_data = pd.read_csv('./output/movie_info.csv')

In [3]:
print(rating_data.shape)
rating_data.head(2)

(4441457, 5)


,Unnamed: 0,movieRank,movieId,score,userId
0,0,1,192150,8,잡는다(jsm6****)
1,1,1,192150,8,추억(king****)


In [4]:
print(movie_data.shape)
movie_data.head(2)

(98, 5)


,Unnamed: 0,movieRank,movieId,score,userId
0,0,1,192150,모가디슈,"액션, 드라마"
1,1,2,184318,블랙 위도우,"액션, 모험, SF"


In [5]:
rating_data.columns = ['index', 'movieRank', 'movieId', 'score', 'userId']
movie_data.columns = ['index', 'movieRank', 'movieId', 'title', 'genres']

In [6]:
rating_data = rating_data.drop(['index', 'movieRank'], axis = 1)
movie_data = movie_data.drop(['index', 'movieRank'], axis = 1)

In [7]:
user_movie_rating = pd.merge(rating_data, movie_data, on = 'movieId')

In [8]:
user_movie_rating.head(3)

,movieId,score,userId,title,genres
0,192150,8,잡는다(jsm6****),모가디슈,"액션, 드라마"
1,192150,8,추억(king****),모가디슈,"액션, 드라마"
2,192150,8,바라는대로(jimi****),모가디슈,"액션, 드라마"


In [9]:
# 아이템 기반 협업 필터링 (item based Collaborative Filtering)
movie_user_rating = user_movie_rating.pivot_table('score', index='title', columns='userId')
# 사용자 기반 협업 필터링 (user based, 그냥 해봄)
user_movie_rating = user_movie_rating.pivot_table('score', index='userId', columns='title')

In [10]:
# 사용자 기반 협업 필터링 (user based, 그냥 해봄)
user_movie_rating.head()

title,007 노 타임 투 다이,강릉,건파우더 밀크셰이크,고백,고장난 론,고질라 VS. 콩,괴기맨숀,귀멸의 칼날: 남매의 연,귀문,그린 나이트,...,콰이어트 플레이스 2,크루엘라,크루즈 패밀리: 뉴 에이지,킬러의 보디가드 2: 킬러의 와이프,톰과 제리,파이프라인,퍼펙트 케어,퍼피 구조대 더 무비,프리 가이,학교 가는 길
userId,,,,,,,,,,,,,,,,,,,,,
00(06__****),NaN,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,...,NaN,NaN,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
00(32wa****),NaN,NaN,10.0,10.0,NaN,NaN,10.0,10.0,10.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,NaN,10.0
00(ahsh****),NaN,NaN,10.0,10.0,NaN,NaN,10.0,10.0,10.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,NaN,10.0
00(dlek****),10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
00(dlth****),NaN,NaN,10.0,10.0,NaN,NaN,10.0,NaN,NaN,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,10.0


In [11]:
# 해볼 것
movie_user_rating.head()

userId,00(06__****),00(32wa****),00(ahsh****),00(dlek****),00(dlth****),00(gkdn****),00(gode****),00(jini****),00(min0****),00(muta****),...,힛(doqg****),힛네(dm20****),힛더짐(zx12****),힛띵(kimx****),힛맨(cant****),힛핫호(hanj****),힝(dold****),힝듀(hana****),힝입니다(ment****),힝힝(gzyq****)
title,,,,,,,,,,,,,,,,,,,,,
007 노 타임 투 다이,NaN,NaN,NaN,10.0,NaN,NaN,NaN,4.0,NaN,NaN,...,7.0,NaN,2.0,NaN,NaN,10.0,9.0,NaN,10.0,NaN
강릉,7.0,NaN,NaN,10.0,NaN,1.0,NaN,4.0,NaN,6.0,...,7.0,NaN,2.0,NaN,6.0,10.0,9.0,10.0,10.0,NaN
건파우더 밀크셰이크,7.0,10.0,10.0,10.0,10.0,1.0,6.0,4.0,10.0,6.0,...,7.0,10.0,2.0,1.0,6.0,10.0,9.0,10.0,10.0,10.0
고백,7.0,10.0,10.0,10.0,10.0,1.0,6.0,4.0,10.0,6.0,...,7.0,10.0,2.0,1.0,6.0,10.0,9.0,10.0,10.0,10.0
고장난 론,7.0,NaN,NaN,10.0,NaN,1.0,6.0,4.0,NaN,6.0,...,7.0,NaN,2.0,NaN,6.0,10.0,9.0,10.0,10.0,NaN


In [12]:
movie_user_rating.fillna(0, inplace = True)
movie_user_rating.head(3)
# 평점을 매기지 않았으면 0, 매겼으면 1~10점의 값을 가짐

userId,00(06__****),00(32wa****),00(ahsh****),00(dlek****),00(dlth****),00(gkdn****),00(gode****),00(jini****),00(min0****),00(muta****),...,힛(doqg****),힛네(dm20****),힛더짐(zx12****),힛띵(kimx****),힛맨(cant****),힛핫호(hanj****),힝(dold****),힝듀(hana****),힝입니다(ment****),힝힝(gzyq****)
title,,,,,,,,,,,,,,,,,,,,,
007 노 타임 투 다이,0.0,0.0,0.0,10.0,0.0,0.0,0.0,4.0,0.0,0.0,...,7.0,0.0,2.0,0.0,0.0,10.0,9.0,0.0,10.0,0.0
강릉,7.0,0.0,0.0,10.0,0.0,1.0,0.0,4.0,0.0,6.0,...,7.0,0.0,2.0,0.0,6.0,10.0,9.0,10.0,10.0,0.0
건파우더 밀크셰이크,7.0,10.0,10.0,10.0,10.0,1.0,6.0,4.0,10.0,6.0,...,7.0,10.0,2.0,1.0,6.0,10.0,9.0,10.0,10.0,10.0


### 아이템 기반 협업 필터링 추천 시스템
- 유사한 아이템끼리 추천 
- 현재는 평점이 데이터로 들어가 있는 상태이기 때문에 평점이 비슷한 아이템을 의미
- 여기서 코사인 유사도(cosine similarity)를 구하면 평점이 비슷한 아이템이 구해짐

In [13]:
item_based_collabor = cosine_similarity(movie_user_rating)
item_based_collabor

array([[1.        , 0.60784931, 0.46926904, ..., 0.49910083, 0.64421796,
        0.46248284],
       [0.60784931, 1.        , 0.77135712, ..., 0.82034678, 0.94351853,
        0.75991022],
       [0.46926904, 0.77135712, 1.        , ..., 0.94018339, 0.72767867,
        0.98502567],
       ...,
       [0.49910083, 0.82034678, 0.94018339, ..., 1.        , 0.77389928,
        0.92613824],
       [0.64421796, 0.94351853, 0.72767867, ..., 0.77389928, 1.        ,
        0.71687137],
       [0.46248284, 0.75991022, 0.98502567, ..., 0.92613824, 0.71687137,
        1.        ]])

In [14]:
item_based_collabor = pd.DataFrame(data = item_based_collabor, index = movie_user_rating.index, columns = movie_user_rating.index)
item_based_collabor.head()

title,007 노 타임 투 다이,강릉,건파우더 밀크셰이크,고백,고장난 론,고질라 VS. 콩,괴기맨숀,귀멸의 칼날: 남매의 연,귀문,그린 나이트,...,콰이어트 플레이스 2,크루엘라,크루즈 패밀리: 뉴 에이지,킬러의 보디가드 2: 킬러의 와이프,톰과 제리,파이프라인,퍼펙트 케어,퍼피 구조대 더 무비,프리 가이,학교 가는 길
title,,,,,,,,,,,,,,,,,,,,,
007 노 타임 투 다이,1.000000,0.607849,0.469269,0.463695,0.572894,0.796729,0.463996,0.488796,0.528116,0.458951,...,0.892269,0.838855,0.613563,0.729289,0.557209,0.560563,0.457304,0.499101,0.644218,0.462483
강릉,0.607849,1.000000,0.771357,0.761926,0.942317,0.762318,0.762463,0.803387,0.868346,0.753975,...,0.681053,0.510090,0.990918,0.833192,0.916631,0.922165,0.751243,0.820347,0.943519,0.759910
건파우더 밀크셰이크,0.469269,0.771357,1.000000,0.987645,0.818377,0.588306,0.988361,0.960050,0.888129,0.977333,...,0.525799,0.393995,0.764288,0.642933,0.841340,0.836313,0.973661,0.940183,0.727679,0.985026
고백,0.463695,0.761926,0.987645,1.000000,0.808372,0.581123,0.999278,0.948225,0.877240,0.989564,...,0.519461,0.389317,0.754931,0.635087,0.831076,0.826096,0.985852,0.928607,0.718771,0.997349
고장난 론,0.572894,0.942317,0.818377,0.808372,1.000000,0.718407,0.808937,0.852398,0.921456,0.799891,...,0.641881,0.480909,0.933750,0.785106,0.972803,0.978662,0.796985,0.870447,0.889022,0.806228


- 자신과 일치하는 값은 유사도 1
- 유사도가 가까울 수록 1에 가까운 값
- 어떤 영화를 시청했을 때(사용자가 어느 영화를 보았을 때) 그 영화가 마음에 들었다면 그것과 비슷한 영화를 추천할 수 있음

In [15]:
def get_item_based_collabor(title):
    return item_based_collabor[title].sort_values(ascending=False)[:6]

In [18]:
get_item_based_collabor('기적')

title
기적                1.000000
고질라 VS. 콩         0.968760
컨저링 3: 악마가 시켰다    0.959919
미션 파서블            0.940443
랑종                0.938098
더 수어사이드 스쿼드       0.922506
Name: 기적, dtype: float64

### 해당 함수의 의미
- 만약 사용자가 '기적'을 보았다고 가정하고 좋은 평점을 매겼다면,
- 이 기적과 유사한 영화를 추천할 수 있음
- => 기적을 보고 좋은 평점을 준 다른 사람들이 또 좋은 점수를 준 영화를 찾는 다는 뜻?